## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()
#data_to_load = os.path.join("WeatherPy_Database.csv")
#city_data_df = pd.read_csv(data_to_load)
#city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Weather
0,0,Hobart,-42.8794,147.3294,59.05,55,75,4.00,AU,broken clouds
1,1,Punta Arenas,-53.1500,-70.9167,46.51,53,75,18.41,CL,broken clouds
2,2,Clyde River,70.4692,-68.5914,1.08,83,94,4.05,CA,overcast clouds
3,3,Airai,-8.9266,125.4092,72.66,56,69,5.14,TL,broken clouds
4,4,Vaini,-21.2000,-175.2000,78.96,83,20,9.22,TO,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Weather
4,4,Vaini,-21.2000,-175.2000,78.96,83,20,9.22,TO,few clouds
6,6,Salalah,17.0151,54.0924,84.29,23,0,12.66,OM,clear sky
11,11,Harper,4.3750,-7.7169,77.22,86,56,3.94,LR,light rain
13,13,Bambous Virieux,-20.3428,57.7575,82.22,69,75,11.50,MU,broken clouds
22,22,Atuona,-9.8000,-139.0333,78.51,73,23,14.65,PF,few clouds
27,27,Barao De Melgaco,-16.1944,-55.9675,79.32,95,99,3.42,BR,overcast clouds
32,32,Cap Malheureux,-19.9842,57.6142,82.47,61,28,8.99,MU,scattered clouds
45,45,Tura,25.5198,90.2201,77.52,51,4,0.94,IN,clear sky
46,46,Kapaa,22.0752,-159.3190,75.18,82,75,3.44,US,broken clouds
49,49,Richards Bay,-28.7830,32.0377,77.25,78,100,14.23,ZA,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID            194
City               194
Lat                194
Lng                194
Max Temp           194
Humidity           194
Cloudiness         194
Wind Speed         194
Country            193
Current Weather    194
dtype: int64

In [6]:
null_row = preferred_cities_df[preferred_cities_df["Country"].isnull()]
null_row

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Weather
513,513,Khorixas,-20.3667,14.9667,80.11,22,54,3.6,NaN,broken clouds


In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()

clean_df.count()

City_ID            193
City               193
Lat                193
Lng                193
Max Temp           193
Humidity           193
Cloudiness         193
Wind Speed         193
Country            193
Current Weather    193
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
4,Vaini,TO,78.96,few clouds,-21.2000,-175.2000,
6,Salalah,OM,84.29,clear sky,17.0151,54.0924,
11,Harper,LR,77.22,light rain,4.3750,-7.7169,
13,Bambous Virieux,MU,82.22,broken clouds,-20.3428,57.7575,
22,Atuona,PF,78.51,few clouds,-9.8000,-139.0333,
27,Barao De Melgaco,BR,79.32,overcast clouds,-16.1944,-55.9675,
32,Cap Malheureux,MU,82.47,scattered clouds,-19.9842,57.6142,
45,Tura,IN,77.52,clear sky,25.5198,90.2201,
46,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,
49,Richards Bay,ZA,77.25,overcast clouds,-28.7830,32.0377,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()


    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.tail()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
707,Khudumelapye,BW,84.07,clear sky,-23.8833,24.7500,
708,Byron Bay,AU,79.72,broken clouds,-28.6500,153.6167,Backpackers Inn on the Beach
712,Jiwani,PK,77.40,scattered clouds,25.0500,61.7417,Al Mumtaz Transport Jiwani
714,Vao,NC,76.77,clear sky,-22.6667,167.4833,Hotel Kou-Bugny
715,Caxito,AO,77.65,light rain,-8.5785,13.6643,


In [11]:
# 7. Drop the rows where there is no Hotel Name.

nan_hotel_df = hotel_df.replace("", float("NaN"), inplace=True)
nan_hotel_df


In [12]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City               181
Country            181
Max Temp           181
Current Weather    181
Lat                181
Lng                181
Hotel Name         181
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))